In [ ]:
import math
import numpy as np
import tomlkit
import pandas as pd
import yaml
from IPython.display import display
import importlib
import copy
import awkward as ak
import seaborn as sns
sns.set_theme()
import matplotlib
import matplotlib.pyplot as plt
import os
from pathlib import Path

import study_lib

In [ ]:
importlib.reload(study_lib)
do_run = study_lib.do_run 
run_experiment = study_lib.run_experiment
config_series = study_lib.config_series

In [ ]:
base_config_yaml = f"""
candidates: 9
voters: 1001
considerations:
- Issues:
    - halfcsep: 0.
      halfvsep: 0.
      sigma: 1.
      sigma_vtr: 2.
    - halfcsep: 0.
      halfvsep: 0.
      sigma: 1.
      sigma_vtr: 2.
methods:
- Plurality:
    strat: Honest
- Plurality:
    strat: Strategic
- InstantRunoff: {{}}
- Range:
    nranks: 2
    strat: Honest
- Range:
    nranks: 2
    strat: Strategic
- STAR:
    strat: Honest
- STAR:
    strat: Strategic
    strategic_stretch_factor: 2.0
- BtrIrv:
    strat: Honest
- RP:
    strat: Honest
- MM:
    strat: Honest
"""
base_config = yaml.safe_load(base_config_yaml)

In [ ]:
sfopts = dict(bbox_inches='tight', transparent=False)

In [ ]:
config = copy.deepcopy(base_config)
df = run_experiment(config_series(config, 'candidates', [3, 5, 8, 13]), trials=50000,
                      with_results=True, show_output=False)
results = df.attrs["results"]
df

In [ ]:
# rslt_5c = do_run(base_config, 10000)
(rslt_2c, rslt_3c, rslt_5c, rslt_8c) = results

In [ ]:
need_cols = ['issues', 'num_smith', 'in_smith', 'methods']  # not using cov_matrix, ideal_cand, or cand_regret for memory savings
rslt_9c = ak.from_parquet('ctr_sqz_2d_9c.parquet', columns=need_cols)
# rslt_5c = ak.from_parquet('ctr_sqz_2d_5c.parquet', columns=need_cols)
# rslt_8c = ak.from_parquet('ctr_sqz_2d_8c.parquet', columns=need_cols)
# rslt_13c = ak.from_parquet('ctr_sqz_2d_13c.parquet', columns=need_cols)

In [ ]:
rslt_9c.fields

In [ ]:
rslt_9c.methods.fields

In [ ]:
rslt=rslt_9c
iss = rslt.issues
ncand = len(rslt.issues[0, :, 0])
fig, axs = plt.subplots(2, 3, sharex="all", sharey="all", squeeze=True, layout="constrained", figsize=(8, 5))
def make_plt(data, ax, label):
    xyrange = [-2, 2]
    ax.hist2d(x=data[:,0].to_numpy(), y=data[:,1].to_numpy(), bins=80, range=[xyrange, xyrange])
    ax.set_xlim(*xyrange)
    ax.set_box_aspect(1)
    ax.set_title(label)
make_plt(iss[np.arange(len(iss)), np.random.randint(0,ncand,len(iss))], axs[0,0], "Candidates")
make_plt(iss[np.arange(len(iss)), rslt.methods.pl_h.winner], axs[0,1], "FPTP")
make_plt(iss[np.arange(len(iss)), rslt.methods.IRV_h.winner], axs[0,2], "RCV")
make_plt(iss[np.arange(len(iss)), rslt.methods.aprv_s.winner], axs[1,0], "Approval")
make_plt(iss[np.arange(len(iss)), rslt.methods.star_6_s.winner], axs[1,1], "STAR")
make_plt(iss[np.arange(len(iss)), rslt.methods.MM_h.winner], axs[1,2], "Minimax")
plt.savefig("plots/twoissue_9c.png", **sfopts)

In [ ]:
ak.count_nonzero(rslt_9c.num_smith < 3) / len(rslt_9c)

In [ ]:
an_issue = {
    "halfcsep": 0.,
    "halfvsep": 0.,
    "sigma": 1.,
    # "sigma_vtr": 1.0,
}

In [ ]:
config = copy.deepcopy(base_config)
config["considerations"][0]["Issues"] = [an_issue] * 2
rslt_2d = do_run(base_config, 40000)

In [ ]:
print("% no Condorcet winner =",
      (1 - ak.count_nonzero(rslt_2d.num_smith < 3) / len(rslt_2d)) * 100)

In [ ]:
config = copy.deepcopy(base_config)
config["considerations"][0]["Issues"] = [an_issue] * 5
rslt_2d = do_run(base_config, 40000)
print("% no Condorcet winner =",
      (1 - ak.count_nonzero(rslt_2d.num_smith < 3) / len(rslt_2d)) * 100)

In [ ]:
config = copy.deepcopy(base_config)
config["voters"] = 31
config["considerations"][0]["Issues"] = [an_issue] * 10
rslt_2d = do_run(base_config, 40000)
print("% no Condorcet winner =",
      (1 - ak.count_nonzero(rslt_2d.num_smith < 3) / len(rslt_2d)) * 100)

In [ ]:
ak.count_nonzero(rslt_5c.in_smith[np.arange(len(iss)), rslt_5c.methods["rp_h"].winner])

In [ ]:
def hist_1d_issue(rslt, method, ax=None, nbins=40, title="", y2lims=None, simple_y2lbls=True):
    ncand = len(rslt.issues[0, :, 0])
    if ax is None:
        _, ax = plt.subplots(1, 1)
    iss = rslt.issues[:,:,0]
    winner_iss = iss[np.arange(len(iss)), rslt.methods[method].winner]
    best_iss = rslt.issues[:, 0, 0]
    winner_hist, bin_edges = np.histogram(winner_iss, bins=nbins, range=(-1,1))
    best_hist, _ = np.histogram(best_iss, bins=nbins, range=(-1,1))
    bin_width = bin_edges[1] - bin_edges[0]
    bin_centers = bin_edges[:-1] + bin_width / 2
    winner_dist_ratio = winner_hist / best_hist
    best_theory = (1. - np.abs(bin_edges))**ncand
    best_theory = np.abs(best_theory[1:] - best_theory[:-1]) / 2 / bin_width
    sns.histplot(winner_iss, binrange=(-1,1), bins=nbins, stat='density', ax=ax, label="winner dist'n")
    ax.plot(bin_centers, best_theory, 'P', markersize=3, c='C1', label="theory, ∞ voters")
    ax.set_xlim([-1, 1])
    ax2 = ax.twinx()
    ax2.semilogy(bin_centers, winner_dist_ratio, '.-', label="winners / max_util", c="C5")
    ax2.set_ylabel("Ratio winner / best candidate")
    if simple_y2lbls:
        ax2.yaxis.set_minor_formatter(matplotlib.ticker.ScalarFormatter())
        ax2.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
    ax2.tick_params(axis='y', which="both", colors='C5')
    #ax2.yaxis.label.set_color("C4")
    if y2lims:
        ax2.set_ylim(y2lims)
    ax.grid(which="major", axis="y")  # is this a bug?
    ax2.grid(which="minor", axis="y")  # or is it this one?
    ax.set_xlabel("Issue position")
    if title:
        plt.title(title)
    h1, l1 = ax.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax.legend(h1+h2, l1+l2, loc="upper right")

In [ ]:
def make_figs(results_, ncands_, column, title, filename, y2lims=None, simple_y2lbls=True):
    for rslt, ncand in zip(results_, ncands_):
        hist_1d_issue(rslt, column, nbins=40, title=title.format(ncand), y2lims=y2lims, simple_y2lbls=simple_y2lbls)
        fn = Path(filename.format(ncand))
        os.makedirs(fn.parent, exist_ok=True)
        plt.savefig(fn, **sfopts)